## Урок 4. Тематическое моделирование. EM-алгоритм
Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

рабочие ссылки с твитами
https://disk.yandex.ru/i/v5HM-ENiGXZVpQ
https://disk.yandex.ru/i/koR5eMCToCZS2Q

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

In [1]:
import pandas as pd
import numpy as np

import re

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

from gensim import corpora
from gensim.models import *

In [2]:
morpher = MorphAnalyzer()

In [3]:
stopwords_list = set(get_stop_words("ru"))

In [4]:
positive = pd.read_csv('data/positive.csv', sep = ';', usecols = [3], names = ['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('data/negative.csv', sep = ';', usecols = [3], names = ['text'])
negative['label'] = ['negative'] * len(negative)

In [5]:
df = pd.concat([positive, negative], ignore_index = True)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [7]:
words_regex = re.compile(r'\w+')

# Формирование токенов длиной более 3 букв
def find_words(text, regex = words_regex):
    tokens = regex.findall(text) 
    return [w for w in tokens if w.isalpha() and len(w)>= 4]

In [8]:
# Удаляем стоп слова и другие символы, не являющимися буквами
def lemmatize(words, lemmer = morpher, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] 
    return [w for w in lemmas if not w in stopwords and w.isalpha()] 

#Чистим от нецензурных слов и слов, не несущих особой смысловой нагрузки
def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens) 
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'значит', ' ',  string_tokens) 
    string_tokens = re.sub(r'почему', ' ',  string_tokens) 
    string_tokens = re.sub(r'свой', ' ',  string_tokens) 
    string_tokens = re.sub(r'капец', ' ',  string_tokens) 
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

In [9]:
df['text'] = df['text'].apply(preprocess)
df.head()

,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


In [10]:
# Словарь уникальных токенов
dictionary = corpora.Dictionary(df['text'])

In [11]:
# Выкидывание редких слов
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
dictionary.save('tweet.dict')
len(dictionary.keys())

8100

In [12]:
# Векторизация твитов
corpus = [dictionary.doc2bow(text) for text in df['text']]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [13]:
# Обработка словаря
mm = corpora.MmCorpus('tweet.model')
print(df['text'].values[0])
print(mm[0])

['школотый', 'поверь', 'самый', 'общество', 'предмет', 'тип']
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]


In [14]:
# Обучение модели
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

In [15]:
# Определяем тематики
lda.show_topics(num_topics=3, num_words=20, formatted=True)

[(0,
  '0.026*"дом" + 0.020*"больш" + 0.018*"новый" + 0.014*"утро" + 0.014*"жаль" + 0.012*"настроение" + 0.011*"урок" + 0.011*"час" + 0.010*"прийти" + 0.010*"мама" + 0.009*"скоро" + 0.009*"найти" + 0.008*"улица" + 0.008*"нравиться" + 0.008*"снег" + 0.007*"твиттер" + 0.007*"нужный" + 0.006*"минута" + 0.006*"фотка" + 0.006*"деньга"'),
 (1,
  '0.036*"завтра" + 0.014*"школа" + 0.014*"друг" + 0.013*"хороший" + 0.013*"последний" + 0.011*"самый" + 0.010*"неделя" + 0.009*"экзамен" + 0.009*"каникулы" + 0.008*"фильм" + 0.008*"казаться" + 0.008*"бояться" + 0.007*"идти" + 0.007*"остаться" + 0.007*"работа" + 0.007*"ужасный" + 0.006*"чувство" + 0.006*"надеяться" + 0.006*"глаз" + 0.006*"маленький"'),
 (2,
  '0.015*"никто" + 0.011*"ночь" + 0.011*"любимый" + 0.011*"голова" + 0.009*"школа" + 0.009*"телефон" + 0.008*"дело" + 0.008*"жалко" + 0.007*"вроде" + 0.007*"хороший" + 0.007*"рука" + 0.007*"боль" + 0.007*"правда" + 0.007*"слово" + 0.007*"вечер" + 0.006*"парень" + 0.006*"уйти" + 0.006*"конец" + 0.006

In [16]:
# Оценка качества модели
print(lda.log_perplexity(corpus))

-8.130031669558306


In [17]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  0.000294558891733034


# Визуализация

In [18]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis

In [19]:
%time v_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(v_data)

CPU times: total: 11.9 s
Wall time: 13.6 s
